In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics
% graphics_toolkit ('qt')
warning('off','all');

rand('state',123456)

executing gpml startup script...


In [2]:
ttcs = 400;
knc = 20; % for each data point consider its k nearest clusters
gamma = 0.3;
sig_temp = 0.1;

In [3]:
nn = 40000; 
nv = 0;
n = nn-nv;
nt = 5730;
load ('./data/protein.mat');
[ndata, ~] = size(M);
X = M(:,2:9);
Y = M(:,1);
R = randperm(ndata);        
ori_xvec_test = X(R(1:nt),:);
ori_yvec_test = Y(R(1:nt));
R(1:nt) = [];
ori_xvec_val = X(R(1:nv),:);          
ori_yvec_val = Y(R(1:nv));
R(1:nv) = [];
ori_xvec = X(R,:);
ori_yvec = Y(R);

In [4]:
ttm = round(n / (ttcs/2));
% fast_induce_step = 50;
induce_step = 100;

xnorm = 1; ynorm = 1;

In [5]:
% input normalization
if xnorm == 1
    norm_xmean = mean(ori_xvec);
    norm_xstd = std(ori_xvec);
    xvec = (ori_xvec - repmat(norm_xmean,n,1)) ./ repmat(norm_xstd,n,1);    
    xvec_test = (ori_xvec_test - repmat(norm_xmean,nt,1)) ./ repmat(norm_xstd,nt,1);
    xvec_val = (ori_xvec_val - repmat(norm_xmean,nv,1)) ./ repmat(norm_xstd,nv,1);
else
    xvec = ori_xvec;
    xvec_test = ori_xvec_test;
    xvec_val = ori_xvec_val;
end

% output normalization
if ynorm == 1
    norm_fmean = mean(ori_yvec);
    norm_fstd = std(ori_yvec);
    yvec = (ori_yvec - norm_fmean) / norm_fstd;
    yvec_test = (ori_yvec_test - norm_fmean) / norm_fstd;
    yvec_val = (ori_yvec_val - norm_fmean) / norm_fstd;
else
    yvec = ori_yvec;
    yvec_test = ori_yvec_test;
    yvec_test_val = ori_yvec_val;
end

In [6]:
testsize = nt; % size of test data
valsize = nv;
induce_step = 100;

In [7]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(xvec,2);
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.Ms = ttm+1;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

meanfunc = [];                    % empty: don't use a mean function
covfunc = opts.covfunc;              % Squared Exponental covariance function
likfunc = opts.likfunc;              % Gaussian likelihood
inffunc = opts.inffunc;
km_iters = 1e4;
opts.induce_step = induce_step ;

In [8]:
% hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];    
opts.numOptFC = 50 ;
opts.Ms = ttm+1;
opts.xvec = xvec;
opts.yvec = yvec;
opts.grbcm_baseline = 0;
opts.global_index = ones(n,1);
opts.inffunc = @infGaussLik; opts.meanfunc = meanfunc; opts.likfunc = likfunc;
opts.covfunc = covfunc;
covfuncF = {@apxSparse, {opts.covfunc}, []};
opts.covfuncF = covfuncF;
opts.compute_hyp = 0;

In [9]:
% default partition
dcs_ecs_r = 0.5;
dcs = round(ttcs*dcs_ecs_r) % size of the communication set
ecs = ttcs - dcs % size of other experts

dcs =  200
ecs =  200


Baselines.

When testing baseline, VFE and SPGP use all partition budget (ttcs = dcs + ecs).

In [10]:
n_per = dcs ; % size of Dc
m = mn = round(n / ecs); % mn is the number of experts (normal)
Indics = randperm(n) ;
I_com = Indics(1:n_per) ; % randomly select communication set
[idx, C] = kmeans(xvec, mn,'MaxIter',km_iters);    
I_com2 = Indics(1:ttcs) ; % randomly select communication set
xu = xvec(I_com2, :);

In [11]:
opts.numOptFC = 30 ;
opts.Ms = mn+1;
opts.xvec = xvec;
opts.yvec = yvec;
opts.induce_size = dcs;
opts.grbcm_baseline = 0;
opts.global_index = ones(n,1);
opts.I_com = I_com;
% opts.inffunc = @infGaussLik; opts.meanfunc = meanfunc; opts.likfunc = likfunc;
opts.covfunc = covfunc;
covfuncF = {@apxSparse, {opts.covfunc}, xvec(I_com,:)};
opts.covfuncF = covfuncF;
opts.compute_hyp = 0;

In [12]:
tic();
% % VFE Baseline
vfe_opts = opts;
vfe_opts.induce_step=100;
vfe_opts.induce_type = 'VFE_opt';
inffunc = @(varargin) infGaussLik(varargin{:}, struct('s', 0.0));
vfe_hyp = struct('mean', [], 'cov', [ones(d,1)*log(ell);log(sqrt(sf2))], 'lik', log(sqrt(sn2)));;
vfe_hyp.xu = xu;
[vfe_hyp, vfe_llhs] = minimize(vfe_hyp,@gp,-vfe_opts.induce_step,inffunc,meanfunc,covfuncF,likfunc,xvec,yvec);
vfe_opts.hyp = vfe_hyp;
vfe_opts.xu = vfe_hyp.xu;
vfe_opts.inffunc = @infGaussLik; vfe_opts.meanfunc = meanfunc; vfe_opts.covfuncF = covfuncF; vfe_opts.likfunc = likfunc;
vfe_opts.covfunc = covfunc;
[tmu, ts2] = gp(vfe_hyp, @infGaussLik, meanfunc, covfuncF, likfunc, xvec, yvec, xvec_test);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[vfeMSE,vfeSMSE,vfeMSLL] = evaluate(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'VFE baseline', n_per, vfeMSE,vfeSMSE,vfeMSLL);
vfe_llh = vfe_llhs(end)
% vfe0_smse_rec(ki) = vfeSMSE; vfe0_msll_rec(ki) = vfeMSLL;
[yu, su] = gp(vfe_hyp, @infGaussLik, meanfunc, covfuncF, likfunc, xvec, yvec, vfe_opts.xu);
vfe_opts.yu = yu; vfe_opts.su = su; 
time_vfe_opt = toc();

Function evaluation    100;  Value 4.414344e+04
VFE baseline (Dc size 200): MSE 18.10876025, SMSE 0.48269818, MSLL -0.36526286
vfe_llh =    4.4143e+04


In [13]:
tic();
sp_opts = opts;
sp_opts.induce_size = ttcs;
sp_opts.induce_type = 'SPGP_opt';
% hyp_init(1:d,1) = -2*opts.hyp.cov(1:d);
% hyp_init(d+1,1) = 2*opts.hyp.cov(d+1);
% hyp_init(d+2,1) = 2*opts.hyp.lik;
hyp_init(1:d,1) = -2*ones(d,1)*log(ell);
hyp_init(d+1,1) = 2*log(sqrt(sf2));
hyp_init(d+2,1) = 2*log(sqrt(sn2));

xu = xvec(I_com2, :);
w_init = [reshape(xu,sp_opts.induce_size*d,1);hyp_init];
% [w,spgp_llhs] = minimize(w_init,'spgp_lik_nohyp',-sp_opts.induce_step,yvec,xvec,sp_opts.induce_size);
[w,spgp_llhs] = minimize(w_init,'spgp_lik',-sp_opts.induce_step,yvec,xvec,sp_opts.induce_size);
xb = reshape(w(1:sp_opts.induce_size*d,1),sp_opts.induce_size,d);
sp_opts.xu = xb;
sp_opts.sp_hyp = w(sp_opts.induce_size*d+1:end,1); % hyp format for SPGP
sp_opts.hyp(1:d,1) = -0.5*sp_opts.sp_hyp(1:d,1); % hyp format for GRBCM
sp_opts.hyp(d+1,1) = 0.5*sp_opts.sp_hyp(d+1,1); 
sp_opts.hyp(d+2,1) = 0.5*sp_opts.sp_hyp(d+2,1); 

[tmu,ts2] = spgp_pred(sp_opts.yvec,sp_opts.xvec,sp_opts.xu,xvec_test,sp_opts.sp_hyp);
if ynorm==1
    tmu = tmu * norm_fstd + norm_fmean;
    ts2 = ts2 * norm_fstd^2;
end
[spgpMSE,spgpSMSE,spgpMSLL] = evaluate(ori_xvec, ori_yvec, ori_xvec_test, ori_yvec_test, tmu, ts2);
fprintf('%s (Dc size %d): MSE %6.8f, SMSE %6.8f, MSLL %6.8f\r\n', 'SPSG baseline', n_per, spgpMSE,spgpSMSE,spgpMSLL);
spgp_llh = spgp_llhs(end)
% spgp0_smse_rec(ki) = spgpSMSE; spgp0_msll_rec(ki) = spgpMSLL;
[yu,su] = spgp_pred(sp_opts.yvec,sp_opts.xvec,sp_opts.xu,sp_opts.xu,sp_opts.sp_hyp);
sp_opts.yu = yu; sp_opts.su = su;
time_spgp_opt = toc;

Function evaluation    100;  Value 3.447922e+04
SPSG baseline (Dc size 200): MSE 17.34319356, SMSE 0.46229161, MSLL -0.52843679
spgp_llh =    3.4479e+04
